In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt


In [2]:
file1 = 'C:/Users/karen/Downloads/kaggle material/bag of words/labeledTrainData.tsv'
file2 = 'C:/Users/karen/Downloads/kaggle material/bag of words/testData.tsv'
file3 = 'C:/Users/karen/Downloads/kaggle material/bag of words/unlabeledTrainData.tsv'


In [3]:
labeledTrain = pd.read_csv(file1,delimiter = '\t')

In [4]:
testData = pd.read_csv(file2,delimiter = '\t')

Data preview

In [5]:
labeledTrain.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [6]:
testData.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


# Data cleaning process

In [7]:
from bs4 import BeautifulSoup

In [8]:
import re # regular expression

In [9]:
from nltk.corpus import stopwords #stop_words 

In [10]:
#define a function to convert the review to text
def prepared_review(review,remove_stopwords):
    #remove html part
    raw_text = BeautifulSoup(review,'lxml').get_text()
    #remove non-letter part
    letters = re.sub('[^a-zA-Z]',' ',raw_text)
    words = letters.lower().split()
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if w not in stop_words]
    return (words)

In [17]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [11]:
X_train = []
for review in labeledTrain['review']:
    X_train.append(' '.join(prepared_review(review,True)))

X_test=[]
for review in testData['review']:
    X_test.append(' '.join(prepared_review(review,True)))

y_train = labeledTrain['sentiment']


features extraction and selection

In [12]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [13]:
#naive bayes model 
from sklearn.naive_bayes import MultinomialNB

In [14]:
#pipeline for process
from sklearn.pipeline import Pipeline
#GridSearchCV for grid search
from sklearn.grid_search import GridSearchCV

C:\Users\karen\Downloads\big tiger\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\karen\Downloads\big tiger\python\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


use CountVectorizer and Tfidfvectorizer for text features extraction


In [15]:

pip_count = Pipeline([('count_vec', CountVectorizer(analyzer='word')), ('mnb', MultinomialNB())])

pip_tfidf = Pipeline([('tfidf_vec', TfidfVectorizer(analyzer='word')), ('mnb', MultinomialNB())])

params_count = {'count_vec__binary':[True, False], 'count_vec__ngram_range':[(1, 1), (1, 2)], 'mnb__alpha':[0.1, 1.0, 10.0]}
params_tfidf = {'tfidf_vec__binary':[True, False], 'tfidf_vec__ngram_range':[(1, 1), (1, 2)], 'mnb__alpha':[0.1, 1.0, 10.0]}

gs_count = GridSearchCV(pip_count, params_count, cv=4, n_jobs=-1, verbose=1)
gs_tfidf = GridSearchCV(pip_tfidf, params_tfidf, cv=4, n_jobs=-1, verbose=1)


In [16]:
gs_count.fit(X_train, y_train)

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.0min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('count_vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preproc...nizer=None, vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'count_vec__binary': [True, False], 'count_vec__ngram_range': [(1, 1), (1, 2)], 'mnb__alpha': [0.1, 1.0, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [17]:
print(gs_count.best_score_)
print(gs_count.best_params_) #figure out the best parameters for the model


0.88216
{'count_vec__binary': True, 'count_vec__ngram_range': (1, 2), 'mnb__alpha': 1.0}


In [18]:
count_y_predict = gs_count.predict(X_test)

In [20]:
gs_tfidf.fit(X_train, y_train)

Fitting 4 folds for each of 12 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.0min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf_vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'tfidf_vec__binary': [True, False], 'tfidf_vec__ngram_range': [(1, 1), (1, 2)], 'mnb__alpha': [0.1, 1.0, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [21]:
print(gs_tfidf.best_score_)
print(gs_tfidf.best_params_)

0.88712
{'mnb__alpha': 0.1, 'tfidf_vec__binary': True, 'tfidf_vec__ngram_range': (1, 2)}


In [22]:
tfidf_y_predict = gs_tfidf.predict(X_test)

In [23]:
#for submission of the data
submission_count = pd.DataFrame({'id': testData['id'], 'sentiment': count_y_predict})

submission_tfidf= pd.DataFrame({'id': testData['id'], 'sentiment': tfidf_y_predict})


submission_count.to_csv('C:/Users/karen/Downloads/kaggle material/bag of words/submission_count.csv', index=False)
submission_tfidf.to_csv('C:/Users/karen/Downloads/kaggle material/bag of words/submission_tfidf.csv', index=False)

In [ ]:
#visualize the result


In [24]:
#unlabeled train data
unlabeled = pd.read_csv(file3,delimiter = '\t',quoting=3)

In [25]:
import nltk.data


In [26]:
#use tokenizer of nltk to split the sentences of reviews
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [27]:
def review_to_sentences(review, tokenizer):
    raw_sentences = tokenizer.tokenize(review.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(prepared_review(raw_sentence, False))
    
    return (sentences)


In [28]:
corpora = []  
#create this for train word vec    
for review in unlabeled['review']:
    corpora += review_to_sentences(review, tokenizer)


C:\Users\karen\Downloads\big tiger\python\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\karen\Downloads\big tiger\python\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\karen\Downloads\big tiger\python\lib\site-packages\bs4\__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beauti

In [29]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 20   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words



INFO: Could not find files for the given pattern(s).


In [ ]:
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(corpora, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = "../Datasets/IMDB/300features_20minwords_10context"
model.save(model_name)

Now we train the model

In [ ]:
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(corpora, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

model.init_sims(replace=True)

model_name = "C:/Users/karen/Downloads/kaggle material/bag of words/300features_20minwords_10context"
model.save(model_name)


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load("C:/Users/karen/Downloads/kaggle material/bag of words/300features_20minwords_10context")
model.most_similar("man")